In [3]:
import gzip 
from numpy.random import choice
import numpy as np 
import time
import tracemalloc

def consume_cpu_ram(n): return np.ones((n, n))
def consume_cpu_ram_128mb(): return consume_cpu_ram(2**12)

# push the process' peak gauge high up and then release the memory
z = [consume_cpu_ram_128mb() for i in range(8)] # 1GB
del z
tracemalloc.start()
a1 = consume_cpu_ram_128mb()
a2 = consume_cpu_ram_128mb()
del a1


start = time.time()
# f = open('GRCh38_latest_genomic.fna','rb')

# content = f.read()

# print(type(f))
# print(f)
# print(type(content))

# print(len(content))
# start_index = content.index(b'NC_000006.12')
# new_content = content[start_index:].replace(b'\n',b'')
# # # print(new_content[:100000])
# start_index2 = new_content.index(b'GATCTT')
# new_content = new_content[start_index2:]
# start_index3 = new_content.index(b'ttggtaccat')

# sequence = new_content[start_index3:start_index+100000]


sequence = open('my_sequence.fna','r').read()

# print(f"Length:{len(sequence)}")
# print(sequence[:100])
# print(len(sequence[:100]))
# print(sequence[-10:])
# print(len(sequence[-10:]))
## 0th order Markov Chain
base_tallies = []
base_tallies.append(sequence.count('a'))
base_tallies.append(sequence.count('g'))    
base_tallies.append(sequence.count('c')) 
base_tallies.append(sequence.count('t'))

base_tallies = np.array(base_tallies).astype(float)

sequence_length = float(len(sequence))

probabilities = base_tallies /sequence_length

print(f'Sum of probs: {np.sum(probabilities)}')
print(f'Probs       : {probabilities}')
probabilities = probabilities/np.sum(probabilities)

print(f'Prob A: {probabilities[0]:0.3f}')
print(f'Prob G: {probabilities[1]:0.3f}')
print(f'Prob C: {probabilities[2]:0.3f}')
print(f'Prob T: {probabilities[3]:0.3f}')

bases = ['a','g','c','t']


output_sequence_zero = []
for i in range(100): 
    output_sequence_zero.append(choice(bases,p=probabilities))

# print('Zero order Output:')
# print(output_sequence_zero)


# my_sequence = sequence[:100000]
# print(f'My Sequence Length: {len(my_sequence)}')
# print(f'My Sequence Start : {my_sequence[:10]}')
# print(f'My Sequence End   : {my_sequence[-10:]}')
# print('Saving my Sequence:')

# my_sequence_file = open('my_sequence.fna','w+')
# my_sequence_file.write(my_sequence)


log2_probability_zero = 0

for base in sequence:
    base_index = bases.index(base)
    log2_probability_zero += np.log2(probabilities[base_index])

print(f'Log2Prob for Zeroth Order: {log2_probability_zero}')


## 1st order Markov Chain

transition_probabilities = np.zeros(shape=(4,4))

# for index1,base1 in enumerate(bases):
#     for index2,base2 in enumerate(bases):
#         transition_probabilities[index1][index2] = sequence.count(base1 + base2)
#         print(f'{base1+base2} has a count of : {transition_probabilities[index1][index2]}')

base_patterns = np.array([
['aa','at','ac','ag'],
['ta','tt','tc','tg'],
['ca','ct','cc','cg'],
['ga','gt','gc','gg']
])

print('First Order Count')
for i in range(len(sequence) - 1):
    if (sequence[i] + sequence[i+1]) in base_patterns:
        x = np.where(base_patterns==(sequence[i]+sequence[i+1]))[0][0]
        y = np.where(base_patterns==(sequence[i]+sequence[i+1]))[1][0]
        transition_probabilities[x][y] += 1
print(transition_probabilities)
for row in transition_probabilities:
    row /= np.sum(row)

print('Probability 1st')
print(transition_probabilities)
print(np.sum(transition_probabilities))
sequence = sequence.casefold()

print(f'Length of sequence:{len(sequence)}')


first_base = choice(bases,p=probabilities)
output_sequence_first = [first_base]

for i in range(100):
    base_index = bases.index(output_sequence_first[-1])
    output_sequence_first.append( choice(bases,p=transition_probabilities[base_index]) )

# print('First Order Output Sequence:')
# print(output_sequence_first)

log2_probability_first = 0

for count,base in enumerate(sequence):
    if count == 0:
        log2_probability_first += np.log2(probabilities[bases.index(base)])
    else:
        index0 = bases.index(sequence[count-1])
        index1 = bases.index(sequence[count])
        log2_probability_first += np.log2(transition_probabilities[index0][index1])


print(f'Log2Prob for First Order: {log2_probability_first}')






# 2nd Order Markov Chain

transition_probabilities_2 = np.zeros(shape=(4,4,4))

base_patterns2 = np.array([
[['aaa','aag','aac','aat' ],['aga','agg','agc','agt' ],['aca','acg','acc','act' ],['ata','atg','atc','att' ]],
[['gaa','gag','gac','gat' ],['gga','ggg','ggc','ggt' ],['gca','gcg','gcc','gct' ],['gta','gtg','gtc','gtt' ]],
[['caa','cag','cac','cat' ],['cga','cgg','cgc','cgt' ],['cca','ccg','ccc','cct' ],['cta','ctg','ctc','ctt' ]],
[['taa','tag','tac','tat' ],['tga','tgg','tgc','tgt' ],['tca','tcg','tcc','tct' ],['tta','ttg','ttc','ttt' ]]
])

for i in range(len(sequence) - 2):
    if (sequence[i] + sequence[i+1] + sequence[i+2]) in base_patterns2:
        x = np.where(base_patterns2==(sequence[i]+sequence[i+1]+sequence[i+2]))[0][0]
        y = np.where(base_patterns2==(sequence[i]+sequence[i+1]+sequence[i+2]))[1][0]
        z = np.where(base_patterns2==(sequence[i]+sequence[i+1]+sequence[i+2]))[2][0]
        transition_probabilities_2[x][y][z] += 1

for square in transition_probabilities_2:
    for row in square:
        row /= np.sum(row)
print('trans prob 2')
print(transition_probabilities_2)
print(np.sum(transition_probabilities_2))

first_base  = choice(bases , p=probabilities)
second_base = choice(bases , p=transition_probabilities[bases.index(first_base)])
output_sequence_second = [first_base,second_base]


for i in range(100):
    base_index_1 = bases.index(output_sequence_second[-1])
    base_index_0 = bases.index(output_sequence_second[-2])
    output_sequence_second.append( choice(bases,p=transition_probabilities_2[base_index_0][base_index_1]) )

# print('Second Order Output Sequence:')
# print(output_sequence_second)

log2_probability_second = 0


for count,base in enumerate(sequence):
    if   count == 0:
        log2_probability_second += np.log2(probabilities[bases.index(base)])
    elif count == 1:
        base_index_0 = bases.index(sequence[count-1])
        base_index_1 = bases.index(sequence[count])
        log2_probability_second += np.log2(transition_probabilities[base_index_0][base_index_1])
    else:
        base_index_0 = bases.index(sequence[count-2])
        base_index_1 = bases.index(sequence[count-1])
        base_index_2 = bases.index(sequence[count])
        log2_probability_second += np.log2(transition_probabilities_2[base_index_0][base_index_1][base_index_2])

print(f'Log2Prob for Second Order: {log2_probability_second}')






#### Hidden Markov Model ####

states = ['A','G','C','T']

state_transition_probabilities = np.array([
    [0.4,0.4,0.1,0.1],
    [0.1,0.4,0.4,0.1],
    [0.1,0.1,0.4,0.4],
    [0.4,0.1,0.1,0.4]
])

state_emission_probabilities = np.array([
    [0.4,0.2,0.2,0.2],
    [0.2,0.4,0.2,0.2],
    [0.2,0.2,0.4,0.2],
    [0.2,0.2,0.2,0.4]
])

starting_state = choice(states)
state_sequence_hidden  = [starting_state]
output_sequence_hidden = [choice(bases,p=state_emission_probabilities[states.index(starting_state)]) ]



for i in range(len(sequence)-1):
    state_index = states.index(state_sequence_hidden[-1])
    new_state   = choice(states,p=state_transition_probabilities[state_index])
    state_sequence_hidden.append(new_state)
    new_output  = choice(bases,p=state_emission_probabilities[states.index(new_state)])
    output_sequence_hidden.append(new_output)


log_2_emission_probabilities   = np.log2(state_emission_probabilities)
log_2_transition_probabilities = np.log2(state_transition_probabilities) 
current_f  = np.array([0,0,0,0])
previous_f = np.array([0,-10000000,-10000000,-10000000])


for base_count,base in enumerate(sequence):
    for current_state_count in range(4):
        emission_probability = log_2_emission_probabilities[current_state_count][bases.index(base)]
        b_array = np.array([0,0,0,0])
        for previous_state_count in range(4):
            b_array[previous_state_count] = previous_f[previous_state_count] + log_2_transition_probabilities[previous_state_count][current_state_count]
        b = np.max(b_array)
        term = 0
        for previous_state_count in range(4):
            term += 2**(previous_f[previous_state_count] + log_2_transition_probabilities[previous_state_count][current_state_count] - b) 
        current_f[current_state_count] = emission_probability + b + np.log2(term)

    if base_count != len(sequence)-1:
        previous_f = current_f

sequence_probability = np.sum(current_f)

print(f'Sequence Log2 Probability is: {sequence_probability}')




# for count,(state,output) in enumerate(zip(state_sequence_hidden,output_sequence_hidden)):
#     if count <100:
#         print(state,output)
#     else: break 

# print('Hidden Markov Output:')
# print('State Path:')
# print(state_sequence_hidden[:50])
# print('Output Sequence:')
# print(output_sequence_hidden[:50])

#Most likely state sequence
from hmmlearn import hmm
model = hmm.MultinomialHMM(n_components=len(states))
model.startprob_ = np.array([0.3, 0.4, 0.2, 0.1]) #self-defined

model.transmat_ = state_transition_probabilities
model.emissionprob_ = state_emission_probabilities

# Predict the optimal sequence of internal hidden state
X = np.atleast_2d([3,4,5,6,7]).T
model.fit(X)
state_sequence = model.predict(X)
prob_next_step = model.transmat_[state_sequence[-1], :]

print(prob_next_step)
print(model.decode(X))

end = time.time() - start
print(f'Running time: {end} seconds')

current, peak = list(map(lambda x: int(x/2**20), tracemalloc.get_traced_memory()))
tracemalloc.stop()
print(f"Memory used {current:0.2f}MB, {peak:0.2f}MB")



Sum of probs: 0.9999999999999999
Probs       : [0.31515 0.20796 0.20803 0.26886]
Prob A: 0.315
Prob G: 0.208
Prob C: 0.208
Prob T: 0.269
Log2Prob for Zeroth Order: -197688.94757657658
First Order Count
[[11197.  7576.  5537.  7205.]
 [ 6241.  7909.  5546.  7190.]
 [ 7796.  6656.  5308.  1043.]
 [ 6281.  4744.  4412.  5358.]]
Probability 1st
[[0.35529113 0.24039346 0.17569411 0.22862129]
 [0.23212825 0.29416797 0.20627836 0.26742543]
 [0.37475364 0.31995385 0.25515551 0.050137  ]
 [0.30204376 0.22813176 0.21216639 0.25765809]]
4.0
Length of sequence:100000
Log2Prob for First Order: -209264.46712988027
trans prob 2
[[[0.41537912 0.19665982 0.15361257 0.23434849]
  [0.33282443 0.24941013 0.20721721 0.21054823]
  [0.42297273 0.05111071 0.22846307 0.29745349]
  [0.28220697 0.25527983 0.17014256 0.29237064]]

 [[0.34819296 0.27479701 0.16143926 0.21557077]
  [0.30851064 0.25531915 0.22956327 0.20660694]
  [0.34088849 0.05847688 0.28286491 0.31776972]
  [0.22533727 0.32630691 0.19709106 0.251

Fitting a model with 43 free scalar parameters with only 5 data points will result in a degenerate solution.


Sequence Log2 Probability is: -845059
[1.00000000e+00 2.67130104e-37 5.40377786e-65 1.62882810e-43]
(-1.399096577156529, array([1, 3, 2, 0, 2]))
Running time: 60.99342918395996 seconds
Memory used 145.00MB, 256.00MB


In [44]:
#self-defined
states = ['A','G','C','T','O']

state_transition_probabilities = np.array([
    [0.5   , 0.125 , 0.125 , 0.125 , 0.125],
    [0.125 , 0.5   , 0.125 , 0.125 , 0.125],
    [0.125 , 0.125 , 0.5   , 0.125 , 0.125],
    [0.125 , 0.125 , 0.125 , 0.5   , 0.125],
    [0.05  , 0.05  , 0.05  , 0.05  , 0.8]
])

state_emission_probabilities = np.array([
    [0.5   , 0.166 , 0.167 , 0.167],
    [0.166 , 0.5   , 0.167 , 0.167],
    [0.167 , 0.166 , 0.5   , 0.167],
    [0.167 , 0.167 , 0.166 , 0.5  ],
    [probabilities[0],probabilities[1],probabilities[2],probabilities[3]]
])

In [45]:
from hmmlearn import hmm
model = hmm.MultinomialHMM(n_components=len(states))
model.startprob_ = np.array([0.3, 0.4, 0.2, 0.1]) #self-defined

model.transmat_ = state_transition_probabilities
model.emissionprob_ = state_emission_probabilities

# Predict the optimal sequence of internal hidden state
X = np.atleast_2d([3,4,5,6,7]).T
model.fit(X)
state_sequence = model.predict(X)
prob_next_step = model.transmat_[state_sequence[-1], :]

print(prob_next_step)
print(model.decode(X))

Fitting a model with 59 free scalar parameters with only 5 data points will result in a degenerate solution.


[2.39832741e-57 7.07467807e-46 1.00000000e+00 4.42904765e-43
 1.59822834e-96]
(-0.00012392416348709856, array([4, 1, 3, 2, 0]))


In [46]:
from hmmlearn import hmm
model = hmm.GaussianHMM(n_components=len(states))
model.startprob_ = np.array([0.3, 0.4, 0.2, 0.1]) #self-defined

model.transmat_ = state_transition_probabilities
model.emissionprob_ = state_emission_probabilities

# Predict the optimal sequence of internal hidden state
X = np.atleast_2d([3,4,5,6,7]).T
model.fit(X)
state_sequence = model.predict(X)
prob_next_step = model.transmat_[state_sequence[-1], :]

print(prob_next_step)
print(model.decode(X))

Fitting a model with 34 free scalar parameters with only 5 data points will result in a degenerate solution.


[1.16246445e-195 0.00000000e+000 1.00000000e+000 1.56947493e-052
 0.00000000e+000]
(6.918232798946864, array([1, 4, 0, 3, 2]))


In [11]:
ai=0
ti=0
ci=0
gi=0
for count in range(len(sequence)):
    if sequence[count] == "a":
        ai+=1
    elif sequence[count] == "t":
        ti+=1
    elif sequence[count] == "c":
        ci+=1
    else:
        gi+=1

print(ai,ti,ci,gi)    

31515 26886 20803 20796


In [18]:
base_patterns

array([['aa', 'ag', 'ac', 'at'],
       ['ga', 'gg', 'gc', 'gt'],
       ['ca', 'cg', 'cc', 'ct'],
       ['ta', 'tg', 'tc', 'tt']], dtype='<U2')

In [4]:
s = ['a', 't','c','g']
order1 = {}
for i in s:
    for j in s:
        order1[i+j]=0
for i in range(len(sequence)-1):
    i1 = sequence[i]
    i2 = sequence[i+1]
    order1[i1+i2]+=1

In [5]:
order1

{'aa': 11197,
 'at': 7576,
 'ac': 5537,
 'ag': 7205,
 'ta': 6241,
 'tt': 7909,
 'tc': 5546,
 'tg': 7190,
 'ca': 7796,
 'ct': 6656,
 'cc': 5308,
 'cg': 1043,
 'ga': 6281,
 'gt': 4744,
 'gc': 4412,
 'gg': 5358}

In [15]:
order2 = {}
for i in s:
    for j in s:
        for k in s:
            order2[i+j+k]=0
for i in range(len(sequence) - 2):
    i1 = sequence[i]
    i2 = sequence[i+1]
    i3 = sequence[i+2]
    order2[i1+i2+i3]+=1

In [16]:
order2

{'aaa': 4651,
 'aat': 2624,
 'aac': 1720,
 'aag': 2202,
 'ata': 2138,
 'att': 2215,
 'atc': 1289,
 'atg': 1934,
 'aca': 2342,
 'act': 1647,
 'acc': 1265,
 'acg': 283,
 'aga': 2398,
 'agt': 1517,
 'agc': 1493,
 'agg': 1797,
 'taa': 2177,
 'tat': 1791,
 'tac': 1147,
 'tag': 1126,
 'tta': 1755,
 'ttt': 2807,
 'ttc': 1756,
 'ttg': 1591,
 'tca': 2019,
 'tct': 1857,
 'tcc': 1447,
 'tcg': 223,
 'tga': 1974,
 'tgt': 1863,
 'tgc': 1454,
 'tgg': 1898,
 'caa': 2182,
 'cat': 1807,
 'cac': 1656,
 'cag': 2151,
 'cta': 1279,
 'ctt': 1694,
 'ctc': 1566,
 'ctg': 2117,
 'cca': 1931,
 'cct': 1750,
 'ccc': 1348,
 'ccg': 279,
 'cga': 256,
 'cgt': 257,
 'cgc': 235,
 'cgg': 295,
 'gaa': 2187,
 'gat': 1354,
 'gac': 1014,
 'gag': 1726,
 'gta': 1069,
 'gtt': 1192,
 'gtc': 935,
 'gtg': 1548,
 'gca': 1504,
 'gct': 1402,
 'gcc': 1248,
 'gcg': 258,
 'gga': 1653,
 'ggt': 1107,
 'ggc': 1230,
 'ggg': 1368}